In [4]:

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import fundamentalanalysis as fa
from datetime import datetime
FA_API_KEY = '85fe259a4ec6fad3cbe55a5ddaf7f9b4'

# save for 3 hours
def load_data(ticker):
    # load data
    profile = fa.profile(ticker, FA_API_KEY)
    key_metrics_annually = fa.key_metrics(ticker, FA_API_KEY, period="annual")
    stock_data = fa.stock_data(ticker, period="5y", interval="1d")
    financial_ratios_annually = fa.financial_ratios(ticker, FA_API_KEY, period="annual")
    income_statement_annually = fa.income_statement(ticker, FA_API_KEY, period="annual")
    try:
        dividends = fa.stock_dividend(ticker, FA_API_KEY)
        dividends.index = pd.to_datetime(dividends.index)
        dividends = dividends["adjDividend"].resample("1Y").sum().sort_index()
    except:
        dividends = pd.Series(0, name="Dividends")

    # return information of interest
    return {
        "stock_closings": stock_data["close"].sort_index(),
        "historical_PE": key_metrics_annually.loc["peRatio"].sort_index(),
        "payout_ratio": financial_ratios_annually.loc["payoutRatio"].sort_index(),
        "dividend_yield": 100*financial_ratios_annually.loc["dividendYield"].sort_index(),
        "cash_per_share": key_metrics_annually.loc["cashPerShare"].sort_index(),
        "debt_to_equity": key_metrics_annually.loc["debtToEquity"].sort_index(),
        "free_cash_flow_per_share": key_metrics_annually.loc["freeCashFlowPerShare"].sort_index(),
        "dividends": dividends,
        "earnings_per_share": income_statement_annually.loc["eps"].sort_index(),
        "info": profile.iloc[:, 0]
    }

In [12]:
stock_data = fa.stock_data("AAPL", period="5y", interval="1d")
stock_data.head()

,low,close,high,open,volume,adjclose
2018-02-23,42.927502,43.125000,43.487499,42.950001,123967600,41.111969
2018-02-24,43.384998,43.875000,43.912498,43.417500,135249600,41.826958
2018-02-27,44.052502,44.742500,44.847500,44.087502,152648800,42.653973
2018-02-28,44.540001,44.597500,45.119999,44.775002,155712400,42.515732
2018-03-01,44.512501,44.529999,45.154999,44.814999,151128400,42.451385


In [18]:
x = stock_data["close"]
x.index

Index([2018-02-23, 2018-02-24, 2018-02-27, 2018-02-28, 2018-03-01, 2018-03-02,
       2018-03-03, 2018-03-06, 2018-03-07, 2018-03-08,
       ...
       2023-02-08, 2023-02-09, 2023-02-10, 2023-02-11, 2023-02-14, 2023-02-15,
       2023-02-16, 2023-02-17, 2023-02-18, 2023-02-22],
      dtype='object', length=1258)

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import fundamentalanalysis as fa
from datetime import datetime
import plotly.express as px
from datetime import date, timedelta
import numpy as np
FA_API_KEY = '85fe259a4ec6fad3cbe55a5ddaf7f9b4'

profile = fa.profile('AAPL', FA_API_KEY)

In [ ]:
dividends = fa.stock_dividend('META', FA_API_KEY)
# dividends.index = pd.to_datetime(dividends.index)
x = {"dividends": dividends}
dividend_dates = x["dividends"]["paymentDate"].values
dividend_dates = dividend_dates[dividend_dates != ""]
def convert_date(date_string):
    return datetime.strptime(date_string, "%Y-%m-%d").date()
date_objects = np.vectorize(convert_date)(dividend_dates)

In [4]:
dividends = 0
def graph(closing, time_frame):
    today = date.today()
    if time_frame == '1 Week':
        start_date = today - timedelta(days = 7)
    elif time_frame == '1 Month':
        start_date = today - timedelta(days = 30)
    elif time_frame == '3 Months':
        start_date = today - timedelta(days = 90)
    elif time_frame == '6 Months':
        start_date = today - timedelta(days = 180)
    elif time_frame == '1 Year':
        start_date = today - timedelta(days = 365)
    elif time_frame == '3 Years':
        start_date = today - timedelta(days = 1095)
    elif time_frame == '5 Years':
        start_date = today - timedelta(days = 1825)
    else:
        raise ValueError("Invalid time frame")
    fig = px.line(x= closing[start_date:today].index, y = closing.loc[start_date:today])
    fig.update_layout(
        yaxis_title = "Price ($)",
        xaxis_title = "Date"
    )
    if dividends != 0:
        past_dates = date_objects[date_objects > start_date]
        dividend_trace = go.Scatter(
            x = past_dates,
            y=closing.loc[past_dates],
            mode='markers',
            marker=dict(
                symbol='cross',
                size=10,
                color='black',
                line=dict(width=2, color='black')
            ),
            name='Dividend Payout Dates'
        )
        fig.add_trace(dividend_trace)
        return fig
    else:
        return fig
prices = fa.stock_data('AAPL', period = "5y", interval = "1d")
graph(prices["close"], '5 Years')

In [142]:
dividends = pd.Series(0, name="Dividends")
dividends

0    0
Name: Dividends, dtype: int64

In [2]:
import plotly.graph_objects as go
payout = x["dividends"]["adjDividend"][0:4]
fig = go.Figure(go.Bar(x = payout.index, y = payout))
fig.update_layout(title = 'Past dividend payout', xaxis_title = 'Date', yaxis_title = 'Payout ($)')
fig

NameError: name 'x' is not defined

In [17]:
dividends= fa.stock_dividend('BIIB', FA_API_KEY)
x = {"div": dividends}
if x["div"].empty== True:
    print('xsdfs')

xsdfs


In [21]:
key_metrics_annually = fa.key_metrics('BIIB', FA_API_KEY, period = 'annual')
key_metrics_annually

,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,...,2000,1999,1998,1997,1996,1995,1994,1993,1992,1991
period,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY,...,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY
revenuePerShare,68.232059,73.653253,83.558732,76.846072,65.65593,57.732361,52.421245,46.65713,41.05333,29.259785,...,2.076181,1.766341,1.259776,0.980066,0.630716,0.693698,0.795061,0.182428,0.150722,0.090963
netIncomePerShare,20.435278,10.43662,24.863891,31.472475,21.623719,11.943086,16.954212,15.374946,12.416637,7.860665,...,0.747547,0.490147,0.313363,0.201338,0.092255,0.025884,-0.024903,0.043333,0.046667,0.010667
operatingCashFlowPerShare,9.284373,24.412475,26.288378,37.833244,30.198633,21.406397,20.70696,16.107932,12.447654,9.898227,...,0.82007,0.605403,0.379218,0.220452,0.105933,0.043211,0.177407,0.063663,0.052515,0.008741
freeCashFlowPerShare,10.896043,22.434608,22.858919,34.254944,25.874085,12.174036,17.375,13.320763,11.230222,-4.912742,...,0.373853,0.413464,0.30328,0.14018,-0.043469,-0.186857,-0.04467,0.049218,0.041062,-0.003704
cashPerShare,32.81556,25.503018,16.221877,23.922501,17.266959,17.351834,22.413462,14.861292,7.807547,5.160958,...,1.529287,1.455298,1.167882,0.993714,0.731532,1.408277,1.361862,0.09964,0.104665,0.083852
bookValuePerShare,89.858484,73.079812,66.502797,71.315874,63.638848,59.326435,55.586538,40.627655,45.731324,36.384764,...,2.479449,2.17788,1.623588,1.210944,1.10253,1.751406,1.677824,0.434938,0.347258,0.354074
tangibleBookValuePerShare,45.160966,19.968478,11.43381,21.667023,20.523182,19.219191,21.295788,11.382748,21.261264,12.296278,...,2.449173,2.14704,1.587735,1.177221,1.078725,1.714876,1.636551,0.425309,0.339339,0.343259
shareholdersEquityPerShare,89.858484,73.079812,66.502797,71.315874,63.638848,59.326435,55.586538,40.627655,45.731324,36.384764,...,2.479449,2.17788,1.623588,1.210944,1.10253,1.751406,1.677824,0.434938,0.347258,0.354074
interestDebtPerShare,44.35949,52.696848,50.035426,35.034206,29.951684,29.111007,31.032051,28.703078,2.600857,2.649851,...,0.244989,0.251379,0.139735,0.206721,0.150847,0.15774,0,0,0,0
